In [1]:
conda list | grep torch

# packages in environment at /cm/shared/apps/miniconda3/envs/torch_env:
torch                       2.9.1            pypi_0           pypi
torchvision                 0.24.1           pypi_0           pypi

Note: you may need to restart the kernel to use updated packages.


In [6]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
               673      defq jupyter-    uat03  R       8:38      1 rsu-training


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# 1. เตรียมข้อมูล (Data Preprocessing)
# หมายเหตุ: ViT และ ResNet มักต้องการภาพขนาด 224x224
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

# 2. การเรียกใช้โมเดลทั้ง 3 รูปแบบ
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- MobileNetV3 (Small) ---
model_mobilenet = models.mobilenet_v3_small(weights='DEFAULT')
model_mobilenet.classifier[3] = nn.Linear(model_mobilenet.classifier[3].in_features, 10) # ปรับให้เข้ากับ 10 คลาสของ CIFAR

# --- ResNet-50 ---
model_resnet = models.resnet50(weights='DEFAULT')
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, 10) # ปรับ Output Layer

# --- Vision Transformer (ViT-B-16) ---
model_vit = models.vit_b_16(weights='DEFAULT')
model_vit.heads.head = nn.Linear(model_vit.heads.head.in_features, 10) # ปรับ Output Layer

# ย้ายโมเดลไปที่ GPU/CPU
model_mobilenet.to(device)
model_resnet.to(device)
model_vit.to(device)

print(f"เตรียมโมเดลบน {device} เรียบร้อยแล้ว!")

/cm/shared/apps/miniconda3/envs/torch_env/lib/python3.11/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


เตรียมโมเดลบน cuda เรียบร้อยแล้ว!


In [3]:
import torch.optim as optim

def train_model(model, train_loader, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch + 1}, Batch {i + 1}: loss {running_loss / 100:.3f}')
                running_loss = 0.0

# ตัวอย่างการเริ่มเทรน MobileNet
# train_model(model_mobilenet, trainloader)

In [4]:
import torch
import time
import torchvision.models as models

# ตั้งค่าอุปกรณ์
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Testing on: {device}")

# 1. โหลดโมเดล
models_dict = {
    "MobileNetV3-Small": models.mobilenet_v3_small(weights='DEFAULT').to(device),
    "ResNet-50": models.resnet50(weights='DEFAULT').to(device),
    "Vision Transformer (ViT-B-16)": models.vit_b_16(weights='DEFAULT').to(device)
}

# 2. สร้างข้อมูลจำลอง (Dummy Input) ขนาด 224x224 ตามมาตรฐาน
dummy_input = torch.randn(1, 3, 224, 224).to(device)

def benchmark(model, name, input_data, iterations=100):
    model.eval()
    # Warm-up (รันก่อนรอบจริงเพื่อให้ระบบเข้าที่)
    with torch.no_grad():
        for _ in range(10):
            _ = model(input_data)

    # เริ่มจับเวลา
    start_time = time.time()
    with torch.no_grad():
        for _ in range(iterations):
            _ = model(input_data)
            if device.type == 'cuda':
                torch.cuda.synchronize() # รอให้ GPU ประมวลผลเสร็จจริง
    end_time = time.time()

    avg_time = (end_time - start_time) / iterations
    fps = 1 / avg_time
    print(f"[{name}] Avg Inference Time: {avg_time*1000:.2f} ms | FPS: {fps:.2f}")

# 3. รันการทดสอบ
print("-" * 50)
for name, model in models_dict.items():
    benchmark(model, name, dummy_input)
print("-" * 50)

Testing on: cuda
--------------------------------------------------
[MobileNetV3-Small] Avg Inference Time: 1.25 ms | FPS: 801.00
[ResNet-50] Avg Inference Time: 2.71 ms | FPS: 369.62
[Vision Transformer (ViT-B-16)] Avg Inference Time: 11.57 ms | FPS: 86.42
--------------------------------------------------


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn

# 1. เตรียมข้อมูลสำหรับทดสอบ (Validation Set)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ใช้ CIFAR-10 เป็นตัวอย่าง (หรือเปลี่ยนเป็น Dataset ของคุณเอง)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. ฟังก์ชันคำนวณ Accuracy
def validate(model, dataloader, model_name):
    model.eval()
    correct_1 = 0
    correct_5 = 0
    total = 0

    print(f"กำลังทดสอบ {model_name}...")
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)

            # Top-1 Accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct_1 += (predicted == labels).sum().item()

            # Top-5 Accuracy
            _, top5_predicted = outputs.topk(5, 1, True, True)
            correct_5 += top5_predicted.eq(labels.view(-1, 1).expand_as(top5_predicted)).sum().item()

    acc1 = 100 * correct_1 / total
    acc5 = 100 * correct_5 / total
    return acc1, acc5

# 3. โหลดโมเดลและปรับแก้ Layer สุดท้าย (กรณีใช้ CIFAR-10 ซึ่งมี 10 คลาส)
def get_model(name):
    if name == "MobileNetV3":
        m = models.mobilenet_v3_small(weights='DEFAULT')
        m.classifier[3] = nn.Linear(m.classifier[3].in_features, 10)
    elif name == "ResNet50":
        m = models.resnet50(weights='DEFAULT')
        m.fc = nn.Linear(m.fc.in_features, 10)
    elif name == "ViT":
        m = models.vit_b_16(weights='DEFAULT')
        m.heads.head = nn.Linear(m.heads.head.in_features, 10)
    return m.to(device)

# 4. รันการเปรียบเทียบ
model_names = ["MobileNetV3", "ResNet50", "ViT"]
results = {}

for name in model_names:
    model = get_model(name)
    # หมายเหตุ: ในการใช้งานจริง คุณต้องโหลด Weight ที่เทรนแล้วมาใส่ก่อน (model.load_state_dict)
    # ไม่อย่างนั้นค่า Accuracy จะเป็นแบบสุ่ม (Random)
    acc1, acc5 = validate(model, testloader, name)
    results[name] = (acc1, acc5)

# 5. แสดงผลสรุปในรูปแบบตาราง
print("\n" + "="*30)
print(f"{'Model':<15} | {'Top-1 Acc':<10} | {'Top-5 Acc':<10}")
print("-" * 30)
for name, acc in results.items():
    print(f"{name:<15} | {acc[0]:>8.2f}% | {acc[1]:>8.2f}%")

กำลังทดสอบ MobileNetV3...
กำลังทดสอบ ResNet50...
กำลังทดสอบ ViT...

Model           | Top-1 Acc  | Top-5 Acc 
------------------------------
MobileNetV3     |    10.24% |    49.32%
ResNet50        |    13.13% |    55.59%
ViT             |     9.40% |    45.15%
